Used to create unique access codes for Lifted Electronic Message Pickup.  Also used to combine PDFs of multiple recipient messages and then rename them with "NetID_accessCode.pdf"

Also, used to format a CSV for MailMerge for sending these messages

In [ ]:
import pandas as pd
import numpy as np
import re
import pypdf
import os

In [ ]:
all = pd.read_csv("all.csv")
all["First Name"] = all["Recipient's First Name"] = all["Recipient\'s Full Name"].str.split(n=1, expand=True)[0]
all

In [ ]:
all_groupby = all.groupby(["Recipient NetID", "Recipient's First Name"])["Recipient NetID"].count().to_frame(name="Num Messages").reset_index()
all_groupby["Email"] = all_groupby["Recipient NetID"] + "@cornell.edu"
all_groupby["Access Code"] = all_groupby.index
all_groupby.to_csv("all_groupby.csv")
all_groupby

In [ ]:
# open the pdf file
reader = pypdf.PdfReader("all.pdf")

# get number of pages
num_pages = len(reader.pages)

# extract text and do the search
for page in reader.pages:
    text = page.extract_text() 
    for netID, accessCode in zip(all_groupby["Recipient NetID"], all_groupby["Access Code"]):
        writer = pypdf.PdfWriter()
        res_search = re.search(netID, text)
        if res_search is not None:
            # print(res_search)

            file_name = netID + "_" + str(accessCode) + ".pdf"

            if os.path.exists("pdfs/" + file_name):
                writer.append("pdfs/" + file_name)

            writer.add_page(page)
            with open("pdfs/" + file_name, 'wb') as out:
                writer.write(out)

Create MailMerge

In [ ]:
post = pd.read_csv("post_lifted_access.csv")
post

In [ ]:
post_merge_list = all_groupby.loc[all_groupby["Recipient NetID"].isin(post["NetID"])]
post_merge_list.to_csv("post_merge_list.csv")
post_merge_list

In [ ]:
# Sanity check (we want this to be empty).  If it's not, either the NetID was not actually lifted or there's a space in the post csv or something
post.loc[~post["NetID"].isin(post_merge_list["Recipient NetID"])]